**[中級機械学習 ホームページ](https://www.kaggle.com/learn/intermediate-machine-learning)**

---


ここからは**欠損値**の扱いについて学んだ知識を実際に試す番だ。

きっと大きな違いを実感できるはず。

# セットアップ

設問は作業内容にフィードバックを返す。
下のセルを実行してフィードバックシステムをセットアップしよう。

In [16]:
# # コードチェック用のセットアップ
# # 採点システムを有効化するためのコード
# # Set up code checking
# from learntools.core import binder
# binder.bind(globals())
# from learntools.ml_intermediate.ex2 import *
# print("Setup Complete")

この演習では、[Kaggle Learnユーザー向け住宅価格コンペティション](https://www.kaggle.com/c/home-data-for-ml-course)のデータを使う。

![Ames Housing データセット画像](https://i.imgur.com/lTJVG4e.png)

次のコードセルはそのまま実行しよう。
`X_train`, `X_valid`, `y_train`, `y_valid` に訓練・検証データが、`X_test` にテストデータが読み込まれる。

In [17]:
# --- データファイルの展開と確認 ---
# !ls ./input/02-missing-values
#   データディレクトリ内のファイル一覧を表示する。
!ls ./input/02-missing-values 
# !echo "---------------"
!echo "---------------"
# !gzip -d -c ./input/02-missing-values/train.csv.gz > ./input/02-missing-values/train.csv
#   train.csv.gz（圧縮ファイル）を解凍してtrain.csvを作成する。
!gzip -d -c ./input/02-missing-values/train.csv.gz > ./input/02-missing-values/train.csv
# !gzip -d -c ./input/02-missing-values/test.csv.gz > ./input/02-missing-values/test.csv
#   test.csv.gz（圧縮ファイル）を解凍してtest.csvを作成する。
!gzip -d -c ./input/02-missing-values/test.csv.gz > ./input/02-missing-values/test.csv
# !ls ./input/02-missing-values
#   再度ファイル一覧を表示し、解凍後のファイルが存在するか確認する。
!ls ./input/02-missing-values 


test.csv  test.csv.gz  train.csv  train.csv.gz


---------------
test.csv  test.csv.gz  train.csv  train.csv.gz


In [18]:
# --- pandas, scikit-learnのインポート ---
import pandas as pd  # データ操作・前処理のためのライブラリ
from sklearn.model_selection import train_test_split  # データ分割用

# --- データの読み込み ---
# train.csv（訓練データ）とtest.csv（テストデータ）をpandasで読み込む。
# index_col='Id' でId列をインデックスとして使う。
X_full = pd.read_csv('./input/02-missing-values/train.csv', index_col='Id')
X_test_full = pd.read_csv('./input/02-missing-values/test.csv', index_col='Id')

# --- 目的変数（SalePrice）と説明変数の分離 ---
# SalePrice（住宅価格）が欠損している行は分析対象外とする。
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
# 目的変数（ターゲット）yにSalePriceを格納。
y = X_full.SalePrice
# 説明変数（特徴量）からSalePrice列を除外。
X_full.drop(['SalePrice'], axis=1, inplace=True)

# --- 数値データのみ抽出 ---
# 機械学習の入門では、まずobject型（文字列など）を除外し、数値型のみで学習するのが簡単。
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# --- 訓練データと検証データへの分割 ---
# train_test_splitで、訓練用（X_train, y_train）と検証用（X_valid, y_valid）に分ける。
# train_size=0.8で8割を訓練、2割を検証に使う。
# random_state=0で分割の再現性を確保。
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

次のコードセルで、訓練データ（X_train）の先頭5行を表示しよう。

pandasの .head() メソッドは、データフレームの最初の5行を簡単に確認できる便利な関数。
データの中身やカラム名、値の型を把握するのに役立つ。

In [19]:
# --- データの先頭5行を表示 ---
# .head() でデータフレームの最初の5行を表示できる。
# データの中身やカラム名、値の型を把握するのに役立つ。
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


先ほど表示したデータの中にも、すでにいくつか欠損値（NaN）が含まれているのが分かる。

次のステップでは、データセット全体の欠損値の状況をより詳しく調べていく。

# ステップ1: 欠損値の事前調査

下のコードセルはそのまま実行しよう。

In [20]:
# --- データの形状と欠損値のカウント ---
# X_train.shape で（行数, 列数）を表示。
# 欠損値（NaN）がどのカラムにどれだけ含まれているかを調べる。
print(X_train.shape)

# 各カラムごとの欠損値の個数を計算。
missing_val_count_by_column = (X_train.isnull().sum())
# 欠損値が1つ以上あるカラムだけを表示。
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [21]:
# --- 欠損値の有無をTrue/Falseで判定 ---
isnull_df = X_train.isnull()
print(isnull_df.head())  # 各セルが欠損値ならTrue, そうでなければFalse

     MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
Id                                                                           
619       False        False    False        False        False      False   
871       False        False    False        False        False      False   
93        False        False    False        False        False      False   
818       False         True    False        False        False      False   
303       False        False    False        False        False      False   

     YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  ...  GarageArea  \
Id                                                     ...               
619         False       False       False       False  ...       False   
871         False       False       False       False  ...       False   
93          False       False       False       False  ...       False   
818         False       False       False       False  ...       False   
303      

上記のコードは、X_trainの各セルが欠損値（NaN）かどうかを判定し、True/Falseのデータフレームを返す。
pandasのisnull()は、各要素がNaNならTrue、そうでなければFalseになる。

In [22]:
# --- 各カラムごとの欠損値の個数をカウント ---
missing_val_count_by_column = isnull_df.sum()
print(missing_val_count_by_column)  # 各カラムごとの欠損値の個数

MSSubClass         0
LotFrontage      212
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         6
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       58
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64


isnull_df.sum() は、各カラムごとにTrue（=1）の数を合計する。
つまり、各カラムに何個の欠損値があるかをSeriesで返す。

In [23]:
# --- 欠損値が1つ以上あるカラムだけを抽出 ---
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


missing_val_count_by_column[missing_val_count_by_column > 0] で、
欠損値が1つ以上あるカラムだけを抽出して表示できる。

### パートA

上の出力結果を使って、以下の質問に答えよう。

- 訓練データの行数はいくつか？
- 欠損値を含むカラムはいくつか？
- 全体で欠損しているデータはいくつか？

（ヒント：print(X_train.shape)や、欠損値カウントの出力をよく見よう）

In [24]:
# # --- 訓練データの行数 ---
# # X_train.shape[0] で行数（サンプル数）が分かる。
# num_rows = 1168  # 行数

# # --- 欠損値を含むカラム数 ---
# # 欠損値が1つ以上あるカラムの数。
# num_cols_with_missing = 3  # 欠損カラム数

# # --- 全体の欠損値の個数 ---
# # 欠損値カウントの合計。
# tot_missing = 276  # 欠損値の総数

# # --- 答え合わせ（Kaggleの自動採点用）---
# step_1.a.check()

In [6]:
# --- ヒントや解答を表示したい場合は、下のコメントアウトを外す ---
# step_1.a.hint()    # ヒントを表示
# step_1.a.solution() # 解答例を表示

### パートB
上記の結果を踏まえて、欠損値への対処として最も良い方法は何だと思う？

+ **自分の答え:** 欠損しているデータが少ないので、削除よりも補完（imputation）が良い選択肢。

In [7]:
# step_1.b.hint()

In [ ]:
# step_1.b.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> Since there are relatively few missing entries in the data (the column with the greatest percentage of missing values is missing less than 20% of its entries), we can expect that dropping columns is unlikely to yield good results.  This is because we'd be throwing away a lot of valuable data, and so imputation will likely perform better.

欠損値への対処法を比較するために、チュートリアルで使った `score_dataset()` 関数をここでも利用する。
この関数はランダムフォレストモデルでの[平均絶対誤差（MAE）](https://en.wikipedia.org/wiki/Mean_absolute_error)を計算する。

In [9]:
# --- モデル評価用関数の定義 ---
from sklearn.ensemble import RandomForestRegressor  # ランダムフォレスト回帰モデル
from sklearn.metrics import mean_absolute_error     # 平均絶対誤差（MAE）

# 様々な前処理方法の性能を比較するための関数。
# ランダムフォレストで学習し、検証データでのMAE（平均絶対誤差）を返す。
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)  # 決定木100本、乱数固定
    model.fit(X_train, y_train)  # 学習
    preds = model.predict(X_valid)  # 予測
    return mean_absolute_error(y_valid, preds)  # MAEを返す

# Step 2: Drop columns with missing values

このステップでは、`X_train` と `X_valid` のデータから欠損値を含むカラムを除去し、
前処理済みのデータフレームを `reduced_X_train` と `reduced_X_valid` に格納する。

In [10]:
# --- 欠損値を含むカラムの除去 ---
# 欠損値（NaN）が1つでも含まれるカラム名をリストアップ。
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()] 

# 上記のカラムを訓練・検証データから除去。
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

# --- 答え合わせ（Kaggleの自動採点用）---
step_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [11]:
# --- ヒントや解答を表示したい場合は、下のコメントアウトを外す ---
# step_2.hint()    # ヒントを表示
# step_2.solution() # 解答例を表示

上記コードでは、まず `X_train` の各カラムについて `isnull().any()` で欠損値が1つでもあるかを判定し、
該当するカラム名をリスト `cols_with_missing` に格納している。

その後、`drop` メソッドでこれらのカラムを訓練・検証データから除去している。

最後に、除去後のカラム数を `shape[1]` で確認している。

【出力例】
```
欠損値を含むカラム: ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
除去後のカラム数: 33
```

---

次のコードセルを実行して、この方法でのMAE（平均絶対誤差）を確認しよう。

In [12]:
# --- 欠損値カラム除去後のモデル評価（MAE算出）---
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))  # MAEを表示

MAE (Drop columns with missing values):
17837.82570776256


# ステップ3: 補完（Imputation）

### パートA

次のコードセルでは、各カラムの平均値で欠損値を補完（impute）する。
補完後のデータフレームは `imputed_X_train` と `imputed_X_valid` に格納する。
カラム名が元の `X_train`, `X_valid` と一致していることを確認しよう。

（補完＝欠損値を何らかの値で埋めること。ここでは平均値で埋める）

In [13]:
# --- 平均値での補完（Imputation）---
from sklearn.impute import SimpleImputer  # 欠損値補完用

# SimpleImputerを使って、各カラムの平均値で欠損値を埋める。
my_imputer = SimpleImputer()  # デフォルトで平均値補完
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))  # 学習データを補完
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))      # 検証データも同じ方法で補完

# SimpleImputerで補完するとカラム名が消えるので、元のカラム名を再設定。
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

# --- 答え合わせ（Kaggleの自動採点用）---
step_3.a.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [14]:
# --- ヒントや解答を表示したい場合は、下のコメントアウトを外す ---
# step_3.a.hint()    # ヒントを表示
# step_3.a.solution() # 解答例を表示

次のコードセルを実行して、この方法でのMAE（平均絶対誤差）を確認しよう。

In [15]:
# --- 平均値補完後のモデル評価（MAE算出）---
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))  # MAEを表示

MAE (Imputation):
18062.894611872147


### パートB

各手法で得られたMAE（平均絶対誤差）を比較しよう。
結果について意外に思うことはあったか？
なぜ一方の手法の方が良い結果になったと思う？

（ヒント：欠損値の扱い方がモデルの性能にどう影響するかを考えてみよう）

In [16]:
# --- ヒントや解答を表示したい場合は、下のコメントアウトを外す ---
# step_3.b.hint()    # ヒントを表示
# step_3.b.solution() # 解答例を表示

In [17]:
step_3.b.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">解説:</span> このデータセットでは欠損値が非常に少ないため、カラムごと削除よりも補完（imputation）の方が良い結果になると予想できる。
しかし実際には、カラムごと削除の方がわずかに良い結果となった！
これはデータのノイズの影響もあるが、補完方法がこのデータセットに最適でない可能性もある。
たとえば、平均値で埋めるよりも0や最頻値、中央値など他の値で埋めた方が良い場合もある。
特に `GarageYrBlt`（ガレージ建築年）のようなカラムでは、欠損値は「ガレージが無い」ことを意味するかもしれない。
この場合、中央値や最小値で埋める方が自然かもしれないし、0で埋めるのは不適切な場合もある。
どの方法が最適かはデータの意味や背景によるので、色々な補完方法を試してみるのが大切。

# ステップ4: テストデータで予測を生成

この最終ステップでは、好きな方法で欠損値を処理してみよう。
訓練・検証データの前処理が終わったら、ランダムフォレストで学習・評価し、
さらにテストデータも同じ方法で前処理して予測値を生成する。
（この予測値はKaggleコンペに提出できる形式）

### パートA

次のコードセルで訓練・検証データを前処理し、
`final_X_train` と `final_X_valid` に格納しよう。
（どんな方法でもOK。下記の条件を満たせば正解になる）
- 前処理後のデータフレームのカラム数が一致していること
- 欠損値が残っていないこと
- `final_X_train`と`y_train`の行数が一致していること
- `final_X_valid`と`y_valid`の行数が一致していること

In [18]:
# --- 訓練・検証データの最終前処理 ---
# ここではSimpleImputerで中央値（median）補完を採用。
# strategy='median' で各カラムの中央値で欠損値を埋める。
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))  # 訓練データを補完
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))      # 検証データも同じ方法で補完

# SimpleImputerで補完するとカラム名が消えるので、元のカラム名を再設定。
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

# --- 答え合わせ（Kaggleの自動採点用）---
step_4.a.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [19]:
# Lines below will give you a hint or solution code
# step_4.a.hint()
# step_4.a.solution()

次のコードセルを実行すると、ランダムフォレストモデルの学習と評価を行う。
（この後テストデータの予測に使うため、ここでは `score_dataset()` 関数は使わない点に注意）

In [20]:
# --- ランダムフォレストで学習・評価 ---
# n_estimators=210 で決定木の本数を増やし、より安定した予測を目指す。
model = RandomForestRegressor(n_estimators=210, random_state=0)
model.fit(final_X_train, y_train)  # 学習

# 検証データで予測し、MAE（平均絶対誤差）を算出。
preds_valid = model.predict(final_X_valid)
print("MAE (Your appraoch):")
print(mean_absolute_error(y_valid, preds_valid))  # MAEを表示

MAE (Your appraoch):
17459.893781256797


### パートB

次のコードセルでテストデータ（X_test）も前処理しよう。
訓練・検証データと同じ方法（ここでは中央値補完）で処理し、
前処理済みのテストデータを `final_X_test` に格納する。

その後、学習済みモデルでテストデータの予測値を `preds_test` に格納する。

（正解判定の条件）
- 欠損値が残っていないこと
- `final_X_test` の行数が `X_test` と一致していること

In [21]:
# --- テストデータの前処理と予測生成 ---
# 訓練・検証データと同じ方法（中央値補完）でテストデータも処理。
final_X_test = pd.DataFrame(final_imputer.transform(X_test))
final_X_test.columns = X_test.columns  # カラム名を再設定

# 学習済みモデルでテストデータの予測値を生成。
preds_test = model.predict(final_X_test)

# preds_test にはテストデータの住宅価格予測値が格納される。
# この値をKaggleコンペに提出することでスコアを得られる。

# --- 答え合わせ（Kaggleの自動採点用）---
step_4.b.check()

In [22]:
# Lines below will give you a hint or solution code
# step_4.b.hint()
# step_4.b.solution()

次のコードセルを実行すると、Kaggleコンペにそのまま提出できる形式で予測結果をCSVファイルに保存できる。

---
【補足解説】
このセルは、テストデータに対する予測結果を 'submission.csv' というファイルに書き出す処理。
Kaggleの提出画面でこのファイルをアップロードすれば、スコアを確認できる。

In [23]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

In [24]:
#for n_estimators in range(180,220):
#    model = RandomForestRegressor(n_estimators=n_estimators, random_state=0)
#    model.fit(final_X_train, y_train)

    # Get validation predictions and MAE
#    preds_valid = model.predict(final_X_valid)
#    print("MAE (Your appraoch):", n_estimators, end=" --> ")
#    print(mean_absolute_error(y_valid, preds_valid))

# Step 5: Submit your results

ステップ4まで完了したら、いよいよKaggleのリーダーボードに結果を提出できる。
（前回の演習でも提出方法を学んだが、忘れた場合は以下の手順を参照）

First, you'll need to join the competition if you haven't already.  So open a new window by clicking on [this link](https://www.kaggle.com/c/home-data-for-ml-course).  Then click on the **Join Competition** button.

![join competition image](https://i.imgur.com/wLmFtH3.png)

次に、以下の手順に従う。
- Begin by clicking on the blue **COMMIT** button in the top right corner.  This will generate a pop-up window.  
- After your code has finished running, click on the blue **Open Version** button in the top right of the pop-up window.  This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
- Click on the **Output** tab on the left of the screen.  Then, click on the **Submit to Competition** button to submit your results to the leaderboard.
- If you want to keep working to improve your performance, select the blue **Edit** button in the top right of the screen. Then you can change your model and repeat the process.

# Keep going

次の章では、**[カテゴリ変数](https://www.kaggle.com/alexisbcook/categorical-variables)** について学び、
それらを機械学習モデルにどう組み込むかを解説する。
カテゴリ変数は実データで非常によく登場するが、前処理せずにモデルへ入力しようとするとエラーになるので注意。

---
**[Intermediate Machine Learning Home Page](https://www.kaggle.com/learn/intermediate-machine-learning)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*